In [1]:
!pip install --upgrade openai

import os
import json
import csv
from getpass import getpass

OPENAI_API_KEY = getpass()
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

from openai import OpenAI # OpenAI official Python package
from IPython.display import Audio # Included to Python

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2
··········


In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# prompt: write a codpiece that analyses img folder, then for each image, send a request to OpenAI endpoint to extract some specific information I will give in the prompt, process result as json and then create a csv and put in results.csv.

import base64
import requests

print("""
Fişlerin bulunduğu klasörün yolunu bulmak için:
1. Sol menüden Dosya ikonuna tıklayın.
2. Yenileme tuşuna tıklayın.
3. Drive > MyDrive klasörünü açarak Google Drive'ınızın ana klasörüne, Drive > Shareddrives klasörüne girerek sizlerle paylaşılan drive'lara göz atabilirsiniz.
4. Fişlerinizin bulunduğu klasöre geldiğinizde klasörün yanındaki Kebap Menü işaretine basınız.
5. Copy path tuşuna basarak yolu kopyalayınız ve aşağıdaki boşluğa yapıştırınız.
""")
# Define the folder containing the images
image_folder = input('Fişlerinizin bulunduğu klasörün yolu: ')

print("""

### BEEEP BOOOP! ÇALIŞIYORUM... ###

""")

# Define the prompt for the OpenAI API call
prompt = """Bu fişteki \"fiş numarası, tarih, işletme ismi, toplam fiyat, kdv oranı \" bilgilerini bir csv satırıymış gibi döndür.

Başlık satırı olmasın yalnızca cevap satırı olsun ve bundan başka hiçbir şey yazdırma.
Sadece row içeriğini yazdır başka hiçbir şey yazdırma. Aşağıdaki kolonların tamamının bulunduğundan emin ol.

CSV Kolonları: "BELGE NO", "BELGE TARIHI", "DEPARTMAN", "BUTCE", "GIDER","HARCAMA YERI","ACIKLAMA","BELGE TURU", "TOPLAM ODENECEK", "KDV ORANI"

- DEPARTMAN, BÜTÇE, GİDER ve AÇIKLAMA'yı her zaman boş bırak. Hiçbir karakter koyma.
- BELGE TÜRÜ'ne FIS yaz.
- HARCAMA YERİ'ne işletme ismini yaz.
- BELGE TARIHI'ni metin biçimnde değil, YYYY-MM-DD olarak tarih biçiminde yaz.
- Toplam Ödenecek kısmını her zaman sayı biçiminde yaz.
- KDV oranı olarak sadece bu seçeneklerden birini seç (%20, %10, %1, %0)

Örnek cevap:
0009,2023-08-07,,,,"X Gıda",,FIS,240.00,%1
"""

# Create a CSV file to store the results
with open("results.csv", "w", newline="") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(["BELGE NO", "BELGE TARIHI", "DEPARTMAN", "BUTCE", "GIDER","HARCAMA YERI","ACIKLAMA","BELGE TURU", "TOPLAM ODENECEK", "KDV ORANI"])

  # Iterate over each image in the folder
  for filename in os.listdir(image_folder):
    if filename.endswith((".jpg", ".png", ".jpeg", ".JPG", ".PNG", ".JPEG")):
      image_path = os.path.join(image_folder, filename)

      filetype = filename.split(".")[-1]

      # Open the image and convert it to base64
      with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()
        image_base64 = base64.b64encode(image_bytes).decode("utf-8")

        response = client.chat.completions.create(
        model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text" :prompt },
                        {
                            "type": "image_url",
                            "image_url":{"url": "data:image/"+filetype+";base64," + image_base64},
                        },
                    ],
                }
            ],
            max_tokens=300,
        )

      print(response.choices[0].message.content)
      # Extract the extracted information from the response
      extracted_information  =response.choices[0].message.content

      if "," in extracted_information:
      # Write the image name and extracted information to the CSV file
        writer.writerow(extracted_information.split(","))
      else:
        print("Problem in: " + filename)

print("""

### BEEEP BOOOP! İŞLEM BAŞARIYLA TAMAMLANDI!

CSV Dosyanızı indirmek için:

1. Sol menüdeki Klasör ikonuna tıklayın.
2. results.csv'nin yanındaki Kebap Menü ikonuna tıklayın.
3. Download tuşuna basın.

### UYARI! Harcamaların doğru geçip geçmediğini insan gözüyle tekrar kontrol ediniz.
""")


Fişlerin bulunduğu klasörün yolunu bulmak için:
1. Sol menüden Dosya ikonuna tıklayın.
2. Yenileme tuşuna tıklayın.
3. Drive > MyDrive klasörünü açarak Google Drive'ınızın ana klasörüne, Drive > Shareddrives klasörüne girerek sizlerle paylaşılan drive'lara göz atabilirsiniz.
4. Fişlerinizin bulunduğu klasöre geldiğinizde klasörün yanındaki Kebap Menü işaretine basınız.
5. Copy path tuşuna basarak yolu kopyalayınız ve aşağıdaki boşluğa yapıştırınız.

Fişlerinizin bulunduğu klasörün yolu: /content/drive/MyDrive/CDR AI/Finans/Ekim/Fişler


### BEEEP BOOOP! ÇALIŞIYORUM... ###


0018,2023-09-15,,,,MiGROS TICARET A.S.,,FIS,812.03,%10
0009,2023-08-07,,,,Cihan Gıda,,FIS,240.00,%1
000095-0031,2022-03-25,,,,,"Kosinitza Cafe",FIS,100.00,%10
0018,2022-07-08,,,,"İstanbul Dudullu Org.San.Bölgesi",,FIS,429.00,%8
15,2022-07-27,,,,KOSİNİTZA CAFE,,FIS,350.00,%10
2,2024-06-17,,,,HODL,,FIS,464.00,%10
7,2024-01-07,,,,HODL,,FIS,478.80,%10
16,2024-01-02,,,,Black Angus,,FIS,529.00,%10
0146,2024-02-07,,,,D&R